# BOT PROJECT
#### NOTEBOOK AIM
This notebook initially acts as notebook / script 2 in the below BOT METHOD #1. See below for details.

## BOT METHOD #1
**Notebook / Script 1:**
1. Call in a websocket price stream from coinbase pro. 
2. Every new minute send close/open (seconds==0) price to a pickle file (dataframe or dictionary)
3. Set a high low price for the given minute and adjust it as the stream sets new records. The low / high value should be reset to the 1m candle open price each minute
4. Every time the high/low changes send the new price to a pickle file


**Notebook / Script 2 - model driven:** 
1. Calls in the passed close/open price every minute to calculate and construct a supertrend dataframe 
2. BUild all model 1, 5, 15m ++ features required of whichever model is currently producing best results
3. Also calls in the low high value passed by notebook 1. 
4. Intra-minute the low high will fluctuate as new records are set
5. Handles the connection to trade platform
6. Executes supertrend buy / sell orders predicted as a win by model.
7. Executes take profits / stops base on the latest high low values passes from the coinbase websocket!

**Possible Problems**
1. The extra time required to pass new high lows from one script to the next could mean a loss of trade value
2. We don't know the problem of having two connections open at once - is this possible even from separate notebooks.
3. Need to solve how to update the supertrend with a dataframe updating every minute

In [ ]:
# !pip install TA-Lib

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas_ta as ta
import talib
from datetime import datetime, timedelta, timezone
import requests
pd.set_option("display.max_columns", None)

In [ ]:
# Import data science libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle

In [ ]:
# Import api libraries
from kucoin_futures.client import Market, Trade
from api import *
key, secret, passphrase = api()

In [ ]:
# Create the trade client
trade=Trade(key=key, secret=secret, passphrase=passphrase, url='https://api-futures.kucoin.com/api/v1/orders')

#### NARRATIVE
In order to get accurate supertrend values on the initial streamed data we need a base dataset that covers the last 4 hours worth of data... this get's us a current supertrend value to within a few cents of the value displayed in trading view - the one our backtest is based upon!!


#### BUILD BASE DATA SET AND CALCULATE HISTORIC SUPERTREND

In [ ]:
# Buy / Sell signal feature 
# If uptrend>0 then buy, elif downtrend>0 then sell, else: NAN

def signal(row):
    """
    In this function we'll build the feature that states whether we're in a buy or sell signal
    """
    
    uptrend = row[8]
    downtrend = row[9]


    if uptrend>0:
        return "buy"
    elif downtrend>0:
        return "sell"
    else:
        return "no_signal"
    

idx=0
def trigger(row, df):
    global idx
    
    signal = row[10]
    
    if idx == 0:
        idx+=1
        return np.nan
    
    elif signal=="sell" and df.loc[idx-1].signal!="sell":
        idx+=1
        return "sellsig"

    elif signal=="buy" and df.loc[idx-1].signal!="buy":
        idx+=1
        return "buysig"
    
    else:
        idx+=1
        return "nosig"

In [ ]:
def call_store(bar_size='60', td=10, multiplier=30, order=False):
    """
    In this function we call in the last 200+ rows of 1,5,15m data
    The bar size, td and multiplier should be set as the following for each
    1m / default
    bar_size='60', td=10, multiplier=30
    5m
    bar_size='300', td=600, multiplier=2
    15m
    bar_size='900', td=2000, multiplier=2
    """
    global idx
    api_url = 'https://api.pro.coinbase.com'
    symbol = 'BTC-USD'

    bar_size = bar_size

    time_end = datetime.now()

    delta = timedelta(minutes=td)
    time_start = time_end - (multiplier*delta)

#     print(time_end, time_start)

    # Put into isoformat for coinbase
    time_start = time_start.isoformat()
    time_end = time_end.isoformat()

    parameters = {
        "start":time_start,
        "end":time_end,
        "granularity": bar_size
    }

    btc_req = requests.get(f"{api_url}/products/{symbol}/candles", 
                         params=parameters, 
                         headers= {"content-type":"application/json"})

    btc = pd.DataFrame(btc_req.json(), 
                      columns=["time", "low", "high", "open", "close", "Volume"]
                     )

    btc.time = pd.to_datetime(btc["time"], unit='s')
    
    # Re-order the features to standard OHLC
    btc = btc[["time", "open", "high", "low", "close", "Volume"]]

    # invert the called in data to carry out supertrend calcs
    btc = btc.loc[::-1]
    
    
    ### IF WE'RE CALLING THIS FUNC FROM A LIVE ORDER - we just want to see the supertrend 
    if order==True:
        # Build the supertrend based on the trading view default parameters
        supertrendi = ta.supertrend(btc["high"],btc["low"], btc["close"], length=10, multiplier=3)

        # Rename the supertrend feature headers for ease of review
        supertrendi.rename(columns={'SUPERT_10_3.0': 'supertrend', 'SUPERTd_10_3.0': 'direction', 
                                    'SUPERTl_10_3.0': 'st_long', 'SUPERTs_10_3.0': 'st_short'
                                   }, inplace=True)

        btc = btc.join(supertrendi)
        btc.reset_index(inplace=True, drop=True)
        
        return btc
        
    
    elif bar_size=='60': ## build the 1m supertrend indicator / trade signal
        # Build the supertrend based on the trading view default parameters
        supertrendi = ta.supertrend(btc["high"],btc["low"], btc["close"], length=10, multiplier=3)

        # Rename the supertrend feature headers for ease of review
        supertrendi.rename(columns={'SUPERT_10_3.0': 'supertrend', 'SUPERTd_10_3.0': 'direction', 
                                    'SUPERTl_10_3.0': 'st_long', 'SUPERTs_10_3.0': 'st_short'
                                   }, inplace=True)

        btc = btc.join(supertrendi)
        btc.reset_index(inplace=True, drop=True)
        
        # Call the signal function and join to the base data
        signal_ = btc.apply(signal, axis='columns').to_frame()
        btc = btc.join(signal_)
        btc.rename(columns={0: 'signal'}, inplace=True)

        # Call the trigger function and join to the master data
        idx=0 # We have to reset our idx counter before running this apply function
        trigger_ = btc.apply(trigger, df=btc, axis="columns").to_frame()
        btc = btc.join(trigger_)
        btc.rename(columns={0: 'trigger'}, inplace=True)
    
    else:
        btc = btc.reset_index(drop=True)
    
    
    
    btc_ind = indicators(btc)
    
    
    return btc_ind

In [ ]:
def indicators(df):
    
    """
    In this function we'll build the indicators necessary for feature engineering and model predictions
    """

    ###### MACD #######
    macd = talib.MACD(df["close"])
    # MACD LINE IS BLUE, SIGNAL LINE IS RED 
    # MACD default parameters are same as trading view
    # cols = ["macd", "macd_hist", "macd_signal"]
    macd_df = pd.DataFrame(macd).T.rename(columns={0:"MACD", 1:"Signal", 2:"Histogram"})


    ###### BBS #########
    bols = talib.BBANDS(df["close"], timeperiod=20, nbdevup=2, nbdevdn=2)
    bols_df = pd.DataFrame(bols).T.rename(columns={0:"upper", 1:"basis", 2:"lower"})
    bols_df = bols_df[["basis", "upper", "lower"]]

    ###### EMAS #######
    ema20 = talib.EMA(df["close"], timeperiod=20)
    ema50 = talib.EMA(df["close"], timeperiod=50)
    ema200 = talib.EMA(df["close"], timeperiod=200)
    ema_df = pd.DataFrame([ema20, ema50, ema200]).T.rename(columns={0:"ema20", 1:"ema50", 2:"ema200"})


    ##### STOCH RSI ##### NOTE WE USE PANDAS TA FOR THIS INDICATOR ###
    stochrsi = ta.stochrsi(df["close"], length=14, rsi_length=14, 
                           k=3, d=3).rename(columns={"STOCHRSIk_14_14_3_3":"K", "STOCHRSId_14_14_3_3":"D"})


    #### RSI #####
    rsi = talib.RSI(df["close"], timeperiod=14)
    rsi = pd.DataFrame(rsi, columns=["RSI"])

    #### VOLUME ####
    # volume = volume_call()
    # volume["vol_ma"] = talib.SMA(volume["volume"], timeperiod=20)
    df["Volume MA"] = talib.SMA(df["Volume"], timeperiod=20)

    ##### JOIN ####
    indicators = macd_df.join(bols_df).join(ema_df).join(stochrsi).join(rsi)


    df = df.join(indicators)
    
    return df


In [ ]:
def feature_eng(btc_one, btc_five, btc_fifteen):
    """
    In this function we'll build the new features for each time frame and also check if the one minute close
    triggers a supertrend signal, then check with the model if we have a confirmed buy ("win") prediction
    or if we ignore the signal??
    """
    
    #### BUILD THE 1M FEATURES
    btc_one["ema_fs"] = btc_one.apply(lambda x: ((x.ema20/x.ema50)*100)-100, axis=1)
    btc_one["ema_svs"] = btc_one.apply(lambda x: ((x.ema50/x.ema200)*100)-100 , axis=1)
    btc_one["ema_fvs"] = btc_one.apply(lambda x: ((x.ema20/x.ema200)*100)-100, axis=1)
    btc_one["ema_sclose"] = btc_one.apply(lambda x: ((x.close/x.ema50)*100)-100, axis=1)
    btc_one["ema_vsclose"] = btc_one.apply(lambda x: ((x.close/x.ema200)*100)-100, axis=1)
    btc_one["bb_ul"] = btc_one.apply(lambda x: ((x.upper/x.lower)*100)-100, axis=1)
    btc_one["bb_cu"] = btc_one.apply(lambda x: ((x.close/x.upper)*100)-100, axis=1)
    btc_one["bbb_cl"] = btc_one.apply(lambda x: ((x.close/x.lower)*100)-100, axis=1)
    btc_one["day_of_week"] = btc_one.apply(lambda x : x.time.day_name(), axis=1)
    btc_one["hour"] = btc_one.time.dt.hour
    
    
    #### BUILD THE 5m FEATURES
    btc_five["ema_fs_5"] = btc_five.apply(lambda x: ((x.ema20/x.ema50)*100)-100, axis=1)
    btc_five["ema_svs_5"] = btc_five.apply(lambda x: ((x.ema50/x.ema200)*100)-100 , axis=1)
    btc_five["ema_fvs_5"] = btc_five.apply(lambda x: ((x.ema20/x.ema200)*100)-100, axis=1)
    btc_five["ema_sclose_5"] = btc_five.apply(lambda x: ((x.close/x.ema200)*100)-100, axis=1)
    btc_five["bb_ul_5"] = btc_five.apply(lambda x: ((x.upper/x.lower)*100)-100, axis=1)
    btc_five["bb_cu_5"] = btc_five.apply(lambda x: ((x.close/x.upper)*100)-100, axis=1)
    btc_five["bbb_cl_5"] = btc_five.apply(lambda x: ((x.close/x.lower)*100)-100, axis=1)

    
    #### BUILD THE 15M FEATURES
    btc_fifteen["ema_fs_15"] = btc_fifteen.apply(lambda x: ((x.ema20/x.ema50)*100)-100, axis=1)
    btc_fifteen["ema_svs_15"] = btc_fifteen.apply(lambda x: ((x.ema50/x.ema200)*100)-100 , axis=1)
    btc_fifteen["ema_fvs_15"] = btc_fifteen.apply(lambda x: ((x.ema20/x.ema200)*100)-100, axis=1)
    btc_fifteen["ema_sclose_15"] = btc_fifteen.apply(lambda x: ((x.close/x.ema200)*100)-100, axis=1)
    btc_fifteen["bb_ul_15"] = btc_fifteen.apply(lambda x: ((x.upper/x.lower)*100)-100, axis=1)
    btc_fifteen["bb_cu_15"] = btc_fifteen.apply(lambda x: ((x.close/x.upper)*100)-100, axis=1)
    btc_fifteen["bbb_cl_15"] = btc_fifteen.apply(lambda x: ((x.close/x.lower)*100)-100, axis=1)
    
    return btc_one, btc_five, btc_fifteen
    

In [ ]:
%%time
btc_one = call_store()
btc_five = call_store(bar_size='300', td=600, multiplier=2)
btc_fifteen = call_store(bar_size='900', td=2000, multiplier=2)

In [ ]:
btc_one, btc_five, btc_fifteen = feature_eng(btc_one, btc_five, btc_fifteen)

### Construct features / select predictors and Merge higher timeframes

In [ ]:
def data_handler():
    
    """
    This function will handle the api call, indicator build, feature eng funcs and also 
    """
    
    # Call in 1, 5, 15m data and build required indicators
    btc_one = call_store()
    btc_five = call_store(bar_size='300', td=600, multiplier=2)
    btc_fifteen = call_store(bar_size='900', td=2000, multiplier=2)
    
    # Call in the feature engineering 
    btc_one, btc_five, btc_fifteen = feature_eng(btc_one, btc_five, btc_fifteen)
    
    
    # Isolate the model features
    one_feats = btc_one[['time', 'close', 'trigger', 'Volume','Volume MA', 'Histogram', 'MACD', 'Signal', 'K', 'D',
                   'ema_fs','ema_svs', 'ema_fvs', 'ema_sclose', 'bb_ul', 'bb_cu', 'bbb_cl',
                   'day_of_week', 'hour']]

    five_feats = btc_five[['time', 'Volume', 'Volume MA', 'RSI', 'Histogram', 'MACD', 'Signal', 'K', 'D', 
                           'ema_fs_5','ema_svs_5', 'ema_fvs_5', 'ema_sclose_5', 'bb_ul_5', 'bb_cu_5', 
                           'bbb_cl_5']]

    fifteen_feats = btc_fifteen[['time', 'Volume','Volume MA', 'RSI', 'Histogram', 'MACD', 'Signal', 
                                 'K', 'D', 'ema_fs_15','ema_svs_15', 'ema_fvs_15', 'ema_sclose_15', 
                             'bb_ul_15', 'bb_cu_15','bbb_cl_15']]
    
    feats = pd.merge_asof(one_feats, five_feats, on="time", direction='backward', suffixes=('', '_five') )

    feats = pd.merge_asof(feats, fifteen_feats, on="time", direction='backward', suffixes=('', '_fifteen'))
    
    return feats

In [ ]:
%%time

feats = data_handler()

In [ ]:
feats.head(2)

In [ ]:
feats_pred = feats.drop(["time", "close"], axis=1)
feats_pred = feats_pred[['Volume', 'Volume MA', 'Histogram', 'MACD', 'Signal', 'K', 'D',
       'ema_fs', 'ema_svs', 'ema_fvs', 'ema_sclose', 'bb_ul', 'bb_cu','bbb_cl', 
       'day_of_week', 'hour', 'trigger' 'Volume_five', 'Volume MA_five', 'RSI',
       'Histogram_five', 'MACD_five', 'Signal_five', 'K_five', 'D_five',
       'ema_fs_5', 'ema_svs_5', 'ema_fvs_5', 'ema_sclose_5', 'bb_ul_5',
       'bb_cu_5', 'bbb_cl_5', 'Volume_fifteen', 'Volume MA_fifteen',
       'RSI_fifteen', 'Histogram_fifteen', 'MACD_fifteen', 'Signal_fifteen',
       'K_fifteen', 'D_fifteen', 'ema_fs_15', 'ema_svs_15', 'ema_fvs_15',
       'ema_sclose_15', 'bb_ul_15', 'bb_cu_15', 'bbb_cl_15']]

In [ ]:
type(feats_pred.hour[0])

In [ ]:
feats_pred.columns

#### Build interface
We need to call in the data at the start of every minute

In [ ]:

def trade_handler(model, feats, signal_flash, time_now, price, tp, trade_prediction):
    
    
    if signal_flash =="buysig":

        # Isolate the prediction features the model is trained to analyse. 
        feats_pred = feats.drop(["time", "close"], axis=1)
        feats_pred = feats_pred[['Volume', 'Volume MA', 'Histogram', 'MACD', 'Signal', 'K', 'D',
                               'ema_fs', 'ema_svs', 'ema_fvs', 'ema_sclose', 'bb_ul', 'bb_cu','bbb_cl', 
                               'day_of_week', 'hour', 'trigger', 'Volume_five', 'Volume MA_five', 'RSI',
                               'Histogram_five', 'MACD_five', 'Signal_five', 'K_five', 'D_five',
                               'ema_fs_5', 'ema_svs_5', 'ema_fvs_5', 'ema_sclose_5', 'bb_ul_5',
                               'bb_cu_5', 'bbb_cl_5', 'Volume_fifteen', 'Volume MA_fifteen',
                               'RSI_fifteen', 'Histogram_fifteen', 'MACD_fifteen', 'Signal_fifteen',
                               'K_fifteen', 'D_fifteen', 'ema_fs_15', 'ema_svs_15', 'ema_fvs_15',
                               'ema_sclose_15', 'bb_ul_15', 'bb_cu_15', 'bbb_cl_15']]
        # Ask the model to predict whether the trade will be succesful or not
        trade_prediction = model.predict(feats_pred.iloc[-1])


    elif signal_flash =="sellsig":

        # Isolate the prediction features the model is trained to analyse. 
        feats_pred = feats.drop(["time", "close"], axis=1)
        feats_pred = feats_pred[['Volume', 'Volume MA', 'Histogram', 'MACD', 'Signal', 'K', 'D',
                               'ema_fs', 'ema_svs', 'ema_fvs', 'ema_sclose', 'bb_ul', 'bb_cu','bbb_cl', 
                               'day_of_week', 'hour', 'trigger', 'Volume_five', 'Volume MA_five', 'RSI',
                               'Histogram_five', 'MACD_five', 'Signal_five', 'K_five', 'D_five',
                               'ema_fs_5', 'ema_svs_5', 'ema_fvs_5', 'ema_sclose_5', 'bb_ul_5',
                               'bb_cu_5', 'bbb_cl_5', 'Volume_fifteen', 'Volume MA_fifteen',
                               'RSI_fifteen', 'Histogram_fifteen', 'MACD_fifteen', 'Signal_fifteen',
                               'K_fifteen', 'D_fifteen', 'ema_fs_15', 'ema_svs_15', 'ema_fvs_15',
                               'ema_sclose_15', 'bb_ul_15', 'bb_cu_15', 'bbb_cl_15']]
        # Ask the model to predict whether the trade will be succesful or not
        trade_prediction = model.predict(feats_pred.iloc[-1])

    if trade_prediction=="win" and signal_flash=="buysig":
        ## place trade - for now place a prxy trade by recording entry price, time, tp
        print(f"{signal_flash} trade opened at {time_now} at {price}$")
        ## PLACE THE BUY TRADE
        trade.create_market_order('XBTUSDM', 'buy', '5', size=100)
        order_live=True
        tp=price*1.0058
        trade_prediction='dormant'

    elif trade_prediction=="win" and signal_flash=="sellsig":
        ## place trade - for now place a prxy trade by recording entry price, time, tp
        print(f"{signal_flash} trade opened at {time_now} at {price}$")
        ## PLACE THE BUY TRADE
        trade.create_market_order('XBTUSDM', 'sell', '5', size=100)
        order_live=True
        tp=price*0.9942
        trade_prediction='dormant'

    elif trade_prediction=="lose":
        # Print a statement to show we have ignored the signal
        print(f"{signal_flash} Trade ignored at {time_now}, model predicted loss!")
        order_live=False
        trade_prediction='dormant'
        ### ADD THIS RESET OF TP ##
        tp = (price*1.0058 if signal_flash=="buysig" else price*0.9942)
        
        
    return trade_prediction, tp, order_live

In [ ]:
# Load model 
model = CatBoostClassifier()
model.load_model("model_cb")

# Set a trade prediction value
trade_prediction="dormant"

# Create a data review switch to tell the loop whether we've checked for a signal or not
data_review = False

# Are we currently in a live order?
order_live = False

# Record the tp
tp=0

while True:

    ### HUNT FOR SUPERTREND TRADE SIGNALS, PASS THROUGH MODEL PREDICTOR, EXECUTE TRADES ###
    if datetime.now().second == 2 and data_review == False and order_live==False:
        
        # Call in the data
        feats = data_handler()
        ## if we get a supertrend trade signal
        signal_flash = feats.iloc[-1].trigger
        
        # Get some details
        time_now = feats.iloc[-1].time
        price = feats.iloc[-1].close
        
        if signal_flash=="buysig" or signal_flash=="sellsig":
            # Send the latest data for model prediction and new trade execution
            trade_prediction, tp, order_live = trade_handler(model=model, 
                                                             feats=feats, 
                                                             signal_flash=signal_flash, 
                                                             time_now=time_now,
                                                             price=price,
                                                             tp=tp,
                                                             trade_prediction=trade_prediction
                                                            )
        
        print(f"tp:{tp}")   
        # Switch the data review to completed
        data_review = True
        print(f"data review complete - close candle at {time_now} price:{price}$")
        
        
    ### HANDLE LIVE TRADES - CLOSE TRADES THAT HIT TPS,  HANDLE REVERSALS
    elif order_live==True:
        # If we're in a live order we want to get access to live prices every 2 seconds to check for target hits
        if datetime.now().second%5==0:
            try:
                live_data = pd.read_csv("prices_test.csv")
            except:
                continue

            # IF WE HAVE A TP HIT CLOSE THE OPEN TRADE
            if signal_flash=="buysig" and live_data.iloc[-1].high>=tp:
                ## CLOSE THE BUY ORDER IN PROFIT
                trade.create_market_order('XBTUSDM', 'buy', '5', closeOrder=True)
                print(f"Buy trade tp hit at {tp}. Trade closed!")
                order_live=False
            elif signal_flash=="sellsig" and live_data.iloc[-1].low<=tp:
                trade.create_market_order('XBTUSDM', 'sell', '5', closeOrder=True)
                order_live=False
                print(f"Sell trade tp hit at {tp}. Trade closed!")
        
        # If we're in a live order but a reversal signal is found
        if datetime.now().second==2 and data_review==False:
            #data = call_store(order=True)
            feats = data_handler()
            
            # Get some details
            time_now = feats.iloc[-1].time
            price = feats.iloc[-1].close
            
            # Buy live but sell signal flashes - close buy open sell
            if signal_flash=="buysig" and feats.iloc[-1].trigger=="sellsig":
                
                # Close the buy order
                trade.create_market_order('XBTUSDM', 'buy', '5', closeOrder=True) ###
                print(f"buy trade stopped out and trade closed at {time_now} price:{price}")
                time.sleep(3)
                # Change the signal flash to sell
                signal_flash="sellsig"
                
                # Send the latest data for model prediction and new trade execution
                trade_prediction, tp, order_live = trade_handler(model=model, 
                                                                 feats=feats, 
                                                                 signal_flash=signal_flash, 
                                                                 time_now=time_now,
                                                                 price=price,
                                                                 tp=tp,
                                                                 trade_prediction=trade_prediction
                                                                )
                print(f"tp:{tp}")
               
            # Sell live but Buy signal flashes - close buy open sell
            elif signal_flash=="sellsig" and feats.iloc[-1].trigger=="buysig":
                
                # Close the sell order
                trade.create_market_order('XBTUSDM', 'sell', '5', closeOrder=True) ###
                print(f"sell trade stopped out and trade closed at {time_now} price:{price}")
                time.sleep(3)
                # Change the signal flash to buy
                signal_flash="buysig"   
                
            
                # Send the latest data for model prediction and new trade execution
                trade_prediction, tp, order_live = trade_handler(model=model, 
                                                                 feats=feats, 
                                                                 signal_flash=signal_flash, 
                                                                 time_now=time_now,
                                                                 price=price,
                                                                 tp=tp,
                                                                 trade_prediction=trade_prediction
                                                                )
                print(f"tp:{tp}")
                

        
            data_review=True
    
    if datetime.now().second > 10 and data_review==True:
        
        # Reset the data review
        data_review = False
        trade_prediction = "dormant"
        print("data review reset")
    
    

In [ ]:
trade.create_market_order('XBTUSDM', 'buy', '5', closeOrder=True)

In [ ]:
%%time
x = call_store(order=True)
x.tail()

if x.iloc[-1].direction==1:
    print("we ok i n a buy")
elif x.iloc[-1].direction==-1:
    print("we need to close a buy")
    print("wee need to open a sell")

In [ ]:
x.tail()

In [ ]:
x.iloc[-1].direction

In [ ]:
%%time
data = call_store(order=True)
feats = data_handler()

In [ ]:
feats.trigger.value_counts()